In [1]:
import igl
import meshplot as mp
import vedo as vd
import numpy as np
from geometry import mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.Optimizer import Optimizer

vd.settings.default_backend = 'k3d'

### Test Circumcircle

In [2]:
p1 = np.array([[0, 0, 0],[1, 0, 0]])
p2 = np.array([[1,-1,3], [1,1,3]])
p3 = np.array([[-2,5,1],[2,1,1]])

c, rad, n = circle_3pts(p1,p2,p3)

circles = []
for ci in range(len(c)):
    #print(f"centers: {c[ci]}\t rad: {rad[ci]}\t n: {n[ci]}")
    if ci%2 == 0:
        circles.append( vd.Circle(c[ci], r=rad[ci], c='red', alpha = 0.5).orientation(n[ci])) 
    else:
        circles.append( vd.Circle(c[ci], r=rad[ci], c="green", alpha = 0.5).orientation(n[ci])) 
pts1 = vd.Points(np.array([p1[0], p2[0], p3[0]]), r = 10, c = 'red')
pts2 = vd.Points(np.array([p1[1], p2[1], p3[1]]), r = 10, c = 'green')

vd.show(pts1, pts2, vd.Points(p3), *circles)


h: [-2.55016352 -0.21128856]
bx: [3.31662479 3.16227766]


AxisError: axis 1 is out of bounds for array of dimension 1

In [2]:
# Create four quads in 3D that merge in a vertex
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

# Make mesh
m = mesh.Mesh()
m.make_mesh(v, fcs)

mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')

# Init constraints
planarity = Planarity()
planarity.initialize_constraint(m)

# init optimizer
opt = Optimizer()

for i in range(10):

    # Add constraints  
    opt.add_constraint("planarity", planarity, m)
    dx = opt.optimize("LM")
    m.vertices += 0.8*dx[:3*m.V].reshape(m.V, 3)
    planarity.normals += 0.8*dx[3*m.V:].reshape(m.F, 3)

    opt.clear_constraints()


mesh2 = vd.Mesh((m.vertices,  fcs), alpha = 0.9, c='b')

vd.show(mesh1, mesh2)



Mesh Data Structure: |V| = 9, |F| = 4, |E| = 12
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)
(39, 39)
(20,)
(39,)


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [28]:
# Create four quads in 3D that merge in a vertex
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

# v = np.array(
#     [
#         [ 0.0,  0.0,  0.8],
#         [ 1.01,  0.03,  0.01],
#         [-1.02,  0.02,  0.01],
#         [ 0.01,  1.1, -0.2],
#         [ 0.01, -1.3, -0.3],
#         [ -0.6,  -1.05,  -0.1]
#     ]
# )

# fcs = np.array([[0, 1, 2, 3],[1, 0, 5, 4]])

new_v = v.copy()


# Make mesh
m = mesh.Mesh()
m.make_mesh(v, fcs)

#print(f"V before:\n{v}")

# Planarity
# Constraint  n . (vi - vj) per each edge (vi, vj) in F, so we have 4 constraints per each face
# Our variables are the vertices of the mesh, so we have 3n variables plus the auxiliary variable
# n the normals of the faces which result in 3*n + 3*F variables
H = np.zeros((m.F*5, m.V*3+ 3*m.F), dtype=np.float64)

b = np.zeros(m.F*5, dtype=np.float64)

# Fill H

# Compute initial auxiliary variable


# Get list of indices 
iv0, iv1, iv2, iv3 = m.faces()[:,0], m.faces()[:,1], m.faces()[:,2], m.faces()[:,3]

# v0, v1, v2, v3 = v[iv0], v[iv1], v[iv2], v[iv3]
diag1 = v[iv2] - v[iv0]
diag2 = v[iv3] - v[iv1]

# # Compute normals
normals = np.cross(diag1, diag2)

#normals = np.repeat([0,0,1], m.F).reshape(m.F, 3)
#l_normals = np.linalg.norm(normals, axis=1)

#normals =np.ones((m.F, 3))

# Variables will be in the form X = [ vix
#                                     viy
#                                     viz
#                                     nix
#                                     niy
#                                     niz]



it = 10

# First Edge Constraint ni . (v1 - v0) = 0 
for i in range(it):
    H = np.zeros((m.F*5, m.V*3+ 3*m.F), dtype=np.float64)
    # # X
    # H[:m.F, 3*iv0] = - normals[:,0]
    # H[:m.F, 3*iv1] =   normals[:,0]
    # # Y
    # H[:m.F, 3*iv0+1] = - normals[:,1]
    # H[:m.F, 3*iv1+1] =   normals[:,1]
    # # Z 
    # H[:m.F, 3*iv0+2] = - normals[:,2]
    # H[:m.F, 3*iv1+2] =   normals[:,2]

    # # Second Edge Constraint ni . (v2 - v1) = 0

    # # X
    # H[m.F:2*m.F, 3*iv1] = - normals[:,0]
    # H[m.F:2*m.F, 3*iv2] =   normals[:,0]
    # # Y
    # H[m.F:2*m.F, 3*iv1+1] = - normals[:,1]
    # H[m.F:2*m.F, 3*iv2+1] =   normals[:,1]
    # # Z
    # H[m.F:2*m.F, 3*iv1+2] = - normals[:,2]
    # H[m.F:2*m.F, 3*iv2+2] =   normals[:,2]

    # # Third Edge Constraint ni . (v3 - v2) = 0

    # # X
    # H[2*m.F:3*m.F, 3*iv2] = - normals[:,0]
    # H[2*m.F:3*m.F, 3*iv3] =   normals[:,0]
    # # Y
    # H[2*m.F:3*m.F, 3*iv2+1] = - normals[:,1]
    # H[2*m.F:3*m.F, 3*iv3+1] =   normals[:,1]
    # # Z
    # H[2*m.F:3*m.F, 3*iv2+2] = - normals[:,2]
    # H[2*m.F:3*m.F, 3*iv3+2] =   normals[:,2]

    # # Fourth Edge Constraint ni . (v0 - v3) = 0

    # # X
    # H[3*m.F:4*m.F, 3*iv3] = - normals[:,0]
    # H[3*m.F:4*m.F, 3*iv0] =   normals[:,0]
    # # Y
    # H[3*m.F:4*m.F, 3*iv3+1] = - normals[:,1]
    # H[3*m.F:4*m.F, 3*iv0+1] =   normals[:,1]
    # # Z
    # H[3*m.F:4*m.F, 3*iv3+2] = - normals[:,2]
    # H[3*m.F:4*m.F, 3*iv0+2] =   normals[:,2]

    # Compute initial auxiliary variable 

    # Compute sums of vertices diferences
    #face_edge_sum = (v1-v0) + (v2-v1) + (v3-v2) + (v0-v3)
    for f in range(m.F):

        iv0, iv1, iv2, iv3 = m.faces()[f]    

        v0, v1, v2, v3 = new_v[iv0], new_v[iv1], new_v[iv2], new_v[iv3]

        # Edge constraint nf(v1 - v0) = 0
        H[        f, 3*iv0: 3*iv0 + 3] = - normals[f]
        H[        f, 3*iv1: 3*iv1 + 3] =   normals[f]

        # Edge constraint nf(v2 - v1) = 0
        H[  m.F + f, 3*iv1: 3*iv1 + 3] = - normals[f]
        H[  m.F + f, 3*iv2: 3*iv2 + 3] =   normals[f]

        # Edge constraint nf(v3 - v2) = 0
        H[2*m.F + f, 3*iv2: 3*iv2 + 3] = - normals[f]
        H[2*m.F + f, 3*iv3: 3*iv3 + 3] =   normals[f]

        # Edge constraint nf(v0 - v3) = 0
        H[3*m.F + f, 3*iv3: 3*iv3 + 3] = - normals[f]
        H[3*m.F + f, 3*iv0: 3*iv0 + 3] =   normals[f]

        # Edge constraint nf(vi - vj) = 0 normal derivative
        H[        f, 3*m.V + 3*f: 3*m.V + 3*f+3] = (v1 - v0)
        H[  m.F + f, 3*m.V + 3*f: 3*m.V + 3*f+3] = (v2 - v1)
        H[2*m.F + f, 3*m.V + 3*f: 3*m.V + 3*f+3] = (v3 - v2)
        H[3*m.F + f, 3*m.V + 3*f: 3*m.V + 3*f+3] = (v0 - v3)

    
        b[        f] = np.dot(normals[f], v1 - v0)
        b[  m.F + f] = np.dot(normals[f], v2 - v1)
        b[2*m.F + f] = np.dot(normals[f], v3 - v2)
        b[3*m.F + f] = np.dot(normals[f], v0 - v3)

        # Unit normal constraint
        H[4*m.F + f, 3*m.V + 3*f: 3*m.V + 3*f+3] = normals[f]
        b[4*m.F + f] = normals[f]@normals[f] - 1


    hess = H.T @ H
    grad = - H.T @ b


    #print(f"normals norms: {np.linalg.norm(normals, axis=1)}")

    print(f"energy: {b @ b}")
    #print(f"Hess: {hess}")
    #print(f"Grad: {grad}")

    diag_H = max(np.diag(hess))
    #print(f"diag_H: {diag_H}")
    hess += np.eye(hess.shape[0]) * 1e-6 * diag_H

    # Solve
    delta = np.linalg.solve(hess, grad)

    step = 0.8

    new_v = new_v + step*delta[:3*m.V].reshape(m.V, 3)
    normals = normals + step*delta[3*m.V:].reshape(m.F, 3)

#print(f"delta:\n{delta[:3*m.V]}")
#print(f"delta:\n{delta[:3*m.V].reshape(m.V, 3)}")
#print(f"v after:\n{v}")
#print(f"v opt:\n{new_v}")

normals_draw = []
# Draw normals
for i in range(m.F):
    iv0, iv1, iv2, iv3 = m.faces()[i]    
    # Compute baricenter
    baricenter = (new_v[iv0] + new_v[iv1] + new_v[iv2] + new_v[iv3])/4
    # Draw normal
    norm_arrow = vd.Arrow(baricenter, baricenter + normals[i], c='g')
    normals_draw.append(norm_arrow)

mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')
mesh2 = vd.Mesh((new_v, fcs), alpha = 0.9, c='b')

print(new_v)
vd.show(mesh1, mesh2)


Mesh Data Structure: |V| = 9, |F| = 4, |E| = 12
energy: 171.85628720367802
energy: 2.03681482140429
energy: 0.43373233036153064
energy: 0.07705210210474875
energy: 0.031784467050890546
energy: 0.009450738303705323
energy: 0.0037967153241796675
energy: 0.0012778421481883686
energy: 0.00047874222838693555
energy: 0.0001682370848151371
[[-0.0282462  -0.03527448  0.54050374]
 [ 1.08241622  0.02345078  0.32472876]
 [-1.07104158  0.09778534  0.20540845]
 [ 0.01338739  1.20236083  0.04901083]
 [ 0.05172566 -1.37693558 -0.12078301]
 [-0.96808993  0.92290748 -0.1020004 ]
 [ 0.96446049  0.93589436 -0.01181976]
 [-1.02053307 -1.00480963 -0.34393629]
 [ 0.99592102 -0.9453791  -0.12111233]]


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [59]:
v, f = igl.read_triangle_mesh("models/catenoid_def_1.obj")
mesh = om.read_trimesh("models/catenoid_def_1.obj")
mp.plot(v, f, c=v[:, 0])

  o catenoid_def


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0485900…

In [57]:
def iso_stereo(n, h):
    # Isotropic model stereographic projection
    # n: normal vector
    # h: distance center to plane
    # return point in I3

    # Concatenate n[1:2] and h
    return np.vstack((n[1:2],v))/(1+ n[2])

In [107]:
a = np.array([[1, 2, 3],[5,4,2] , [9,7,2]])
b = np.array([19,22,89])
# Add b as a column to a
c = np.column_stack((a,b))
print(a[:, 0:2])

[[1 2]
 [5 4]
 [9 7]]


In [121]:
# Get face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Get indices v1 of each face
iv1 = f[:, 0]
# Get vertices of idv1
v1 = v[iv1, :]
# Compute dot product of each face normal with v1
h = np.sum(nf * v1, axis=1)

# Get Isotropic point per face
nv = np.column_stack((nf[:, 0:2], h)) / (1 + nf[:, 2, None])

nf = np.empty((0, 3), dtype=np.int32)

for fi in range(len(f)):
    aux = np.empty((0, ), dtype=np.int32)
    for adf in mesh.ff(mesh.face_handle(fi)):
        aux = np.hstack((aux, [adf.idx()]))
    if len(aux) == 3:
        nf = np.vstack((nf, aux))

mp.plot(nv, nf[:2])




Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.8548016…

In [5]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

v, f = igl.read_triangle_mesh("models/Hall.obj")

# Draw the mesh
n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
c = np.linalg.norm(n, axis=1)
p = mp.plot(v, f, c, return_plot=True) # plot

# Compute Cuvature properties
v1, v2, k1, k2 = igl.principal_curvature(v, f)



# Compute focal surfaces
rho1 = 100
rho2 = 1/k2

step = 0.001

lamb = 0.1 * np.ones(v.shape[0], dtype=np.float64)

J = np.eye(v.shape[0], dtype=np.float64) 

# Precalculate intermediate states
vs = [v]
cs = [c]
for i in range(30):
    # Compute the gradient of the focal surface
    H = csr_matrix(J.T@J)
    b = - J.T@(lamb - rho1)
    delta = spsolve(H, b)
    lamb = lamb + step * delta
    # Update the mesh
    v = v + lamb[:,None]*n

    # Recompute the normals
    n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
    c = np.linalg.norm(n, axis=1)
    # Add to the list
    # p.update_object(vertices=v, colors=c)
    vs.append(v)
    cs.append(c)
    

# Add interactive visulization
@mp.interact(level=(0, 9))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])vs.append(v)
    cs.append(c)
p

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.867670…

interactive(children=(IntSlider(value=0, description='level', max=9), Output()), _dom_classes=('widget-interac…